In [1]:
!python -m spacy download ru_core_news_sm  # no option to install by pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 546.2 kB/s eta 0:00:0000:0100:01
  Using cached pymorphy3-1.2.0-py3-none-any.whl (55 kB)
  Using cached pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl (8.4 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [1]:
import spacy
import pytextrank
from spacy.lang.ru.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [3]:
stopwords = list(STOP_WORDS)
punctuation += '\n'

In [2]:
nlp = spacy.load("ru_core_news_sm")
nlp.add_pipe("textrank", last=True)

In [4]:
text = '''Сколько статей было написано на хабре от обучающих курсов, как тот или иной человек вошел в ИТ, несмотря на то, что он сам юрист и ему уже за 40 лет. Такие статьи вселяют надежду, как передающиеся из уст в уста рассказы о чудотворцах, которые могли исцелять одним касанием.
Не буду оспаривать реалистичность таких вхождений в ИТ... Но давайте зададимся одним вопросом, а какое соотношение таких "чудо-людей" к обычным смертным - тем, кто не прошел в ИТ? Для этого достаточно почитать про понятие "систематическая ошибка выжившего", но тому, кому этого не достаточно, давайте вместе разберемся в цифрах.
Отовсюду мы слышим одно и тоже: нехватка ИТ специалистов, высокие зарплаты, удаленная работа, поддержка ИТ на уровне государства. Звучит шикарно. Возникает ощущение, что это бездонная сфера, которая будет востребована, ну хотя бы вечность, не меньше.
Отступление: ничто не вечно... Инженеры, юристы, экономисты - всех их объединяет, то что они имели высокую популярность в свое время, но время это ушло...
Давайте посмотрим немного повнимательнее на эту сферу ИТ, выбросим всю эту СМИ мишуру, снимем пару ведер лапши с ушей.
У меня есть знакомые, которые хотели войти в ИТ, но так и не вошли. Мое личное соотношение примерно 1 вошедший на 7 не вошедших. Конкуренция очень жесткая.'''

In [5]:
doc = nlp(text)

In [10]:
# var 1
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies:
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [11]:
max_frequency = max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word] / max_frequency

sentence_tokens = [sent for sent in doc.sents]

sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.text.lower()]

In [20]:
sentence_scores

{Сколько статей было написано на хабре от обучающих курсов, как тот или иной человек вошел в ИТ, несмотря на то, что он сам юрист и ему уже за 40 лет.: 1.7142857142857137,
 Такие статьи вселяют надежду, как передающиеся из уст в уста рассказы о чудотворцах, которые могли исцелять одним касанием.: 1.4285714285714282,
 Не буду оспаривать реалистичность таких вхождений в ИТ...: 0.8571428571428571,
 Но давайте зададимся одним вопросом, а какое соотношение таких "чудо-людей" к обычным смертным - тем, кто не прошел в ИТ?: 1.4285714285714282,
 Для этого достаточно почитать про понятие "систематическая ошибка выжившего", но тому, кому этого не достаточно, давайте вместе разберемся в цифрах.: 1.714285714285714,
 Отовсюду мы слышим одно и тоже: нехватка ИТ специалистов, высокие зарплаты, удаленная работа, поддержка ИТ на уровне государства.: 1.4285714285714282,
 Звучит шикарно.: 0.14285714285714285,
 Возникает ощущение, что это бездонная сфера, которая будет востребована, ну хотя бы вечность, не

In [23]:
len(sentence_tokens)

14

In [15]:
select_length = int(len(sentence_tokens) * 0.3)
summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)

In [16]:
summary

[Инженеры, юристы, экономисты - всех их объединяет, то что они имели высокую популярность в свое время, но время это ушло...,
 Для этого достаточно почитать про понятие "систематическая ошибка выжившего", но тому, кому этого не достаточно, давайте вместе разберемся в цифрах.,
 Сколько статей было написано на хабре от обучающих курсов, как тот или иной человек вошел в ИТ, несмотря на то, что он сам юрист и ему уже за 40 лет.,
 Такие статьи вселяют надежду, как передающиеся из уст в уста рассказы о чудотворцах, которые могли исцелять одним касанием.]

In [8]:
# var 2
for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=2):
    print(str(sent))

Давайте посмотрим немного повнимательнее на эту сферу ИТ, выбросим всю эту СМИ мишуру, снимем пару ведер лапши с ушей.

Сколько статей было написано на хабре от обучающих курсов, как тот или иной человек вошел в ИТ, несмотря на то, что он сам юрист и ему уже за 40 лет.
